In [3]:
import pandas as pd 
import pandas as pd
import torch

from functools import partial
from torch import nn
from torch.utils.data import DataLoader

from nba_betting_ai.model.bayesian import BayesianResultPredictor
from nba_betting_ai.training.dataset import NBADataset
from nba_betting_ai.training.pipeline import prepare_data
from nba_betting_ai.data.processing import add_prefix

In [5]:
team_stats_season = [
    'season_games',
    'season_wins', 'season_pts_for', 'season_pts_against',
    'season_wins_avg', 'season_pts_for_avg', 'season_pts_against_avg',
]
team_stats_last_5 = [
    'last_5_wins',
    'last_5_pts_for_avg', 'last_5_pts_against_avg',
    'last_5_pts_for_total', 'last_5_pts_against_total',
]
target_general = ['away_win', 'score_final_diff']
target_team = 'score_final'
identification_cols = [
    'game_id', 'home_team_abbreviation', 'away_team_abbreviation',
]
other_features = [
    'home_score', 'away_score', 'time_remaining', 'score_diff'
]

prefix_home = partial(add_prefix, prefix='home', return_type='list')
prefix_away = partial(add_prefix, prefix='away', return_type='list')
all_features = identification_cols \
            + target_general \
            + prefix_home([target_team]) + prefix_away([target_team]) \
            + prefix_home(team_stats_season) + prefix_away(team_stats_season) \
            + prefix_home(team_stats_last_5) + prefix_away(team_stats_last_5)


In [25]:
data_params = {
    'seasons': 1,
    'seed': 66,
    'test_size': 0.2,
    'n': 20,
    'frac': None
}

feature_cols = [
    'away_season_wins_avg', 'home_season_wins_avg',
    'away_season_pts_diff_avg', 'away_last_5_pts_diff_avg',
    'home_season_pts_diff_avg', 'home_last_5_pts_diff_avg'
]
target_general = ['final_score_diff']

params = {
    'learning_rate': 0.005,
    'lr_decay': 0.99,
    'batch_size': 128,
    'epochs': 10,
}

model_config = {
    'embedding_dim': 8,
    'team_hidden_dim': 32,
    'team_layers': 2,
    'res_hidden_dim': 32,
    'res_layers': 2,
    'time_scaling': True
}

In [30]:
config = {
    'features': feature_cols,
    'target': target_general,
    'training': {
        'params': params,
        'model_config': model_config
    }
}

data = prepare_data(**data_params)
train_dataset = NBADataset(feature_cols, target_general, data.X_train, data.teams)
test_dataset = NBADataset(feature_cols, target_general, data.X_test, data.teams)

train_loader = DataLoader(train_dataset, batch_size=params['batch_size'], shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=params['batch_size'], shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BayesianResultPredictor(
    team_count=len(data.teams),
    team_features=len(feature_cols)//2,
    **model_config
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])
const = 0.5 * torch.log(2 * torch.tensor(torch.pi, dtype=torch.float64)).to(device)

for epoch in range(params['epochs']):
    model.train()
    for i, data in enumerate(train_loader):
        data = {
            k: v.to(device)
            for k, v in data.items()
        }
        target = data.pop('y')
        optimizer.zero_grad()
        output = model(**data)
        mu, logvar = torch.chunk(output, 2, dim=-1)
        logvar = torch.clamp(logvar, min=-10, max=10)
        loss = const + logvar + (target - mu)**2 / torch.exp(logvar)
        loss = torch.mean(loss)
        loss.backward()
        optimizer.step()
        if i % 100 == 0:
            print(f'Epoch {epoch}, iteration {i}, loss: {loss.item()}')
        learning_rate = learning_rate * params.get('lr_decay', 1.0)
        

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for i, data in enumerate(test_loader):
        data = {
            k: v.to(device)
            for k, v in data.items()
        }
        target = data.pop('y')
        output = model(**data)
        mu, a = torch.chunk(output, 2, dim=-1)
        predicted = mu >= 0
        
        total += target.size(0)
        correct += (predicted == target).sum().item()

Epoch 0, iteration 0, loss: 35170408309237.64
Epoch 0, iteration 100, loss: 111.22494950518345
Epoch 0, iteration 200, loss: 10.719148237112844
Epoch 0, iteration 300, loss: 10.840657154290172
Epoch 0, iteration 400, loss: 10.959729530017647
Epoch 0, iteration 500, loss: 10.77739941600765
Epoch 0, iteration 600, loss: 10.942427944499148
Epoch 0, iteration 700, loss: 10.939377909763587
Epoch 0, iteration 800, loss: 10.939305602108739
Epoch 0, iteration 900, loss: 10.945909113580115
Epoch 0, iteration 1000, loss: 10.934320140084933
Epoch 0, iteration 1100, loss: 10.759384246785395
Epoch 0, iteration 1200, loss: 143335.97257447382
Epoch 0, iteration 1300, loss: 106378.65348069805
Epoch 0, iteration 1400, loss: 175484.2121340087
Epoch 0, iteration 1500, loss: 135305.46831568453
Epoch 0, iteration 1600, loss: 121071.23220117428
Epoch 0, iteration 1700, loss: 161790.76566289747
Epoch 0, iteration 1800, loss: 15.461259672591796
Epoch 0, iteration 1900, loss: 11.32309884770233
Epoch 0, iterati

TypeError: ResultPredictor.forward() got an unexpected keyword argument 'y'